# Sentiment Analysis Tools

Lots of libraries exist that will do sentiment analysis for you. Imagine that: just taking a sentence, throwing it into a library, and geting back a score! How convenient!

It also might be **totally irresponsible** unless you know how the sentiment analyzer was built. In this section we're going to see how sentiment analysis is done with a few different packages.

## Installation

Use `pip install` two language processing packages, NLTK and Textblob.

In [2]:
!pip install nltk textblob

     |████████████████████████████████| 1.5 MB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 636 kB 5.3 MB/s eta 0:00:01
     |████████████████████████████████| 303 kB 9.7 MB/s eta 0:00:01
     |████████████████████████████████| 76 kB 5.1 MB/s eta 0:00:011
     |████████████████████████████████| 285 kB 1.5 MB/s eta 0:00:01


## Tools

### NLTK: Natural Language Tooklit

[Natural Language Toolkit](https://www.nltk.org/) is the basis for a lot of text analysis done in Python. It's old and terrible and slow, but it's just been used for so long and does so many things that it's generally the default when people get into text analysis. The new kid on the block is [spaCy](https://spacy.io/), but it doesn't do sentiment analysis out of the box so we're leaving it out of this right now.

When you first run NLTK, you need to download some datasets to make sure it will be able to do everything you want.

In [4]:
import nltk

nltk.download('vader_lexicon')
nltk.download('movie_reviews')
nltk.download('punkt')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/nisha/nltk_data...
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /Users/nisha/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
[nltk_data] Downloading package punkt to /Users/nisha/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

To do sentiment analysis with NLTK, it only takes a couple lines of code. To determine sentiment, it's using a tool called **VADER**.

In [5]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

sia = SIA()
sia.polarity_scores("This restaurant was great, but I'm not sure if I'll go there again.")

{'neg': 0.153, 'neu': 0.688, 'pos': 0.159, 'compound': 0.0276}

Asking `SentimentIntensityAnalyzer` for the `polarity_score` gave us four values in a dictionary:

- **negative:** the negative sentiment in a sentence
- **neutral:** the neutral sentiment in a sentence
- **positive:** the postivie sentiment in the sentence
- **compound:** the aggregated sentiment. 
    
Seems simple enough!

### Use NLTK/VADER to determine the sentiment of the following sentences:

* I just got a call from my boss - does he realise it's Saturday?
* I just got a call from my boss - does he realise it's Saturday? :)
* I just got a call from my boss - does he realise it's Saturday? 😊

Do the results seem reasonable? What does VADER do with emoji and emoticons?

In [6]:
sia = SIA()
sia.polarity_scores("I just got a call from my boss - does he realise it's Saturday?")

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

In [7]:
sia = SIA()
sia.polarity_scores("I just got a call from my boss - does he realise it's Saturday? :)")

{'neg': 0.0, 'neu': 0.786, 'pos': 0.214, 'compound': 0.4588}

In [8]:
sia = SIA()
sia.polarity_scores("I just got a call from my boss - does he realise it's Saturday? 😊")

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

<i> The analysis of the first sentence is accurate but VADER didn't work well with the last two sentences. It seems to have interpreted the emotican as a colon and a right parenthesis, and ignored the emoji altogether.</i>

Why do you think it doesn't understand the emoji the same way it understood the emoticon?

<i>Because it is an image and not text.</i>

#### When VADER was a baby

As we talked about in class, knowing the dataset a language model was trained on can be pretty important!

[Can you uncover how VADER was trained by reading its homepage?](https://github.com/cjhutto/vaderSentiment)

<i>It was trained on orginal lexicon with over 9000 token features and rated by 10 independent human raters.</i>

### TextBlob

TextBlob is built on top of NLTK, but is infinitely easier to use. It's still slow, but _it's so so so easy to use_. 

You can just feed TextBlob your sentence, then ask for a `.sentiment`!

In [25]:
from textblob import TextBlob
from textblob import Blobber
from textblob.sentiments import NaiveBayesAnalyzer

In [26]:
blob = TextBlob("This restaurant was great, but I'm not sure if I'll go there again.")
blob.sentiment

Sentiment(polarity=0.275, subjectivity=0.8194444444444444)

**How could it possibly be easier than that?!?!?** This time we get a `polarity` and a `subjectivity` instead of all of those different scores, but it's basically the same idea.

Try the TextBlob sentiment tool with another sentence of your own.

In [35]:
blob = TextBlob("I'm busy but I can make time for you.")
blob.sentiment

Sentiment(polarity=0.1, subjectivity=0.3)

If you like options: it turns out TextBlob actually has multiple sentiment analysis tools! How fun! We can plug in a different analyzer to get a different result.

In [36]:
blobber = Blobber(analyzer=NaiveBayesAnalyzer())

blob = blobber("This restaurant was great, but I'm not sure if I'll go there again.")
blob.sentiment

Sentiment(classification='pos', p_pos=0.5879425317005774, p_neg=0.41205746829942275)

Wow, that's a **very different result.** To understand why it's so different, we need to talk about where these sentiment numbers come from. You can read about [the library behind TextBlob's opinions about sentiment](https://github.com/clips/pattern/wiki/pattern-en#sentiment) but they don't really go into (easily-accessible) detail about how it happens.

But first: try it with one of your own sentences!

In [37]:
blobber = Blobber(analyzer=NaiveBayesAnalyzer())

blob = blobber("Going to the hair salon is therapeutic.")
blob.sentiment

Sentiment(classification='neg', p_pos=0.2349826782826531, p_neg=0.765017321717346)

## How were they made?

The most important thing to understand is **sentiment is always just an opinion.** In this case it's an opinion, yes, but specifically **the opinion of a machine.**

### VADER

NLTK's Sentiment Intensity Analyzer works is using something called **VADER**, which is a list of words that have a sentiment associated with each of them.

|Word|Sentiment rating|
|---|---|
|tragedy|-3.4|
|rejoiced|2.0|
|disaster|-3.1|
|great|3.1|

If you have more positives, the sentence is more positive. If you have more negatives, it's more negative. It can also take into account things like capitalization - you can read more about the classifier [here](http://t-redactyl.io/blog/2017/04/using-vader-to-handle-sentiment-analysis-with-social-media-text.html), or the actual paper it came out of [here](http://comp.social.gatech.edu/papers/icwsm14.vader.hutto.pdf).

**How do they know what's positive/negative?** They came up with a very big list of words, then asked people on the internet and paid them one cent for each word they scored.

### TextBlob's `.sentiment`

TextBlob's sentiment analysis is based on a separate library called [pattern](https://www.clips.uantwerpen.be/pattern).

> The sentiment analysis lexicon bundled in Pattern focuses on adjectives. It contains adjectives that occur frequently in customer reviews, hand-tagged with values for polarity and subjectivity.

Same kind of thing as NLTK's VADER, but it specifically looks at words from customer reviews.

**How do they know what's positive/negative?** They look at (mostly) adjectives that occur in customer reviews and hand-tag them.

### TextBlob's `.sentiment` + NaiveBayesAnalyzer

TextBlob's other option uses a `NaiveBayesAnalyzer`, which is a machine learning technique. When you use this option with TextBlob, the sentiment is coming from "an NLTK classifier trained on a movie reviews corpus."

**How do they know what's positive/negative?** Looked at movie reviews and scores using machine learning, the computer _automatically learned_ what words are associated with a positive or negative rating.

## What's this mean for me?

When you're doing sentiment analysis with tools like this, you should have a few major questions: 

* Where kind of dataset does the list of known words come from?
* Do they use all the words, or a selection of the words?
* Where do the positive/negative scores come from?

Let's compare the tools we've used so far.

|technique|word source|word selection|scores|
|---|---|---|---|
|NLTK (VADER)|everywhere|hand-picked|internet people, word-by-word|
|TextBlob|product reviews|hand-picked, mostly adjectives|internet people, word-by-word|
|TextBlob + NaiveBayesAnalyzer|movie reviews|all words|automatic based on score|

A major thing that should jump out at you is **how different the sources are.**

While VADER focuses on content found everywhere, TextBlob's two options are specific to certain domains. The [original paper for VADER](http://comp.social.gatech.edu/papers/icwsm14.vader.hutto.pdf) passive-aggressively noted that VADER is effective at general use, but being trained on a specific domain can have benefits: 

> While some algorithms performed decently on test data from the specific domain for which it was expressly trained, they do not significantly outstrip the simple model we use.

They're basically saying, "if you train a model on words from a certain field, it will be good at sentiment in that certain field."

### Comparison chart

Because they're build differently, sentiment analysis tools don't always agree. Let's take a set of sentences and compare each analyzer's understanding of them.

In [38]:
import pandas as pd
pd.set_option("display.max_colwidth", 200)

df = pd.DataFrame({'content': [
    "I love love love love this kitten",
    "I hate hate hate hate this keyboard",
    "I'm not sure how I feel about toast",
    "Did you see the baseball game yesterday?",
    "The package was delivered late and the contents were broken",
    "Trashy television shows are some of my favorites",
    "I'm seeing a Kubrick film tomorrow, I hear not so great things about it.",
    "I find chirping birds irritating, but I know I'm not the only one",
]})
df

/Users/nisha/.pyenv/versions/3.8.10/lib/python3.8/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


,content
0,I love love love love this kitten
1,I hate hate hate hate this keyboard
2,I'm not sure how I feel about toast
3,Did you see the baseball game yesterday?
4,The package was delivered late and the contents were broken
5,Trashy television shows are some of my favorites
6,"I'm seeing a Kubrick film tomorrow, I hear not so great things about it."
7,"I find chirping birds irritating, but I know I'm not the only one"


In [39]:
def get_scores(content):
    blob = TextBlob(content)
    nb_blob = blobber(content)
    sia_scores = sia.polarity_scores(content)
    
    return pd.Series({
        'content': content,
        'textblob': blob.sentiment.polarity,
        'textblob_bayes': nb_blob.sentiment.p_pos - nb_blob.sentiment.p_neg,
        'nltk': sia_scores['compound'],
    })

scores = df.content.apply(get_scores)
scores.style.background_gradient(cmap='RdYlGn', axis=None, low=0.4, high=0.4)

,content,textblob,textblob_bayes,nltk
0,I love love love love this kitten,0.500000,-0.087933,0.957100
1,I hate hate hate hate this keyboard,-0.800000,-0.214151,-0.941300
2,I'm not sure how I feel about toast,-0.250000,0.394659,-0.241100
3,Did you see the baseball game yesterday?,-0.400000,0.613050,0.000000
4,The package was delivered late and the contents were broken,-0.350000,-0.574270,-0.476700
5,Trashy television shows are some of my favorites,0.000000,0.040076,0.421500
6,"I'm seeing a Kubrick film tomorrow, I hear not so great things about it.",0.800000,0.717875,-0.629600
7,"I find chirping birds irritating, but I know I'm not the only one",-0.200000,0.257148,-0.250000


Wow, those really don't agree with one another! Which one do you agree with the most? Did it get everything "right?"

While it seemed like magic to be able to plug a sentence into a sentiment analyzer and get a result back... maybe things aren't as magical as we thought.

#### Try ten sentences of your own

Just curious: can you make sentences that specifically "trick" one sentiment analysis tool or another?

In [40]:
df = pd.DataFrame({'content': [
    "It takes everything in me just to get up each day",
    "But it’s wonderful to see that you’re okay",
    "Hello, Mr. perfectly fine",
    "How’s your heart after breaking mine?",
    "Mr. always at the right place at the right time",
    "I’ve been Miss misery for the last time"
]})
df

,content
0,It takes everything in me just to get up each day
1,But it’s wonderful to see that you’re okay
2,"Hello, Mr. perfectly fine"
3,How’s your heart after breaking mine?
4,Mr. always at the right place at the right time
5,I’ve been Miss misery for the last time


In [41]:
scores = df.content.apply(get_scores)
scores.style.background_gradient(cmap='RdYlGn', axis=None, low=-0.4, high=0.4)

,content,textblob,textblob_bayes,nltk
0,It takes everything in me just to get up each day,0.000000,0.380663,0.000000
1,But it’s wonderful to see that you’re okay,0.750000,0.449124,0.812600
2,"Hello, Mr. perfectly fine",0.416667,0.442021,0.718400
3,How’s your heart after breaking mine?,0.000000,0.065808,0.000000
4,Mr. always at the right place at the right time,0.285714,0.355556,0.000000
5,I’ve been Miss misery for the last time,0.000000,0.186652,-0.648600


## Review

**Sentiment analysis** is judging whether a piece of text has positive or negative emotion. We covered several tools for doing automatic sentiment analysis: **NLTK**, and two techniques inside of **TextBlob**.

Each tool uses a different data to determine what is positive and negative, and while some use **humans** to flag things as positive or negative, others use a automatic **machine learning**.

As a result of these differences, each tool can come up with very **different sentiment scores** for the same piece of text.

## Discussion topics

The first questions are about whether an analyzer can be applied in situations other than where it was trained. Among other things, you'll want to think about whether the language it was trained on is similar to the language you're using it on.

**Is it okay to use a sentiment analyzer built on product reviews to check the sentiment of tweets?** How about to check the sentiment of wine reviews?

<i>I don't think it would accurately analyze tweets because it can be tricked by sarcasm. Wine reviews would be a better fit.</i>

**Is it okay to use a sentiment analyzer trained on everything to check the sentiment of tweets?** How about to check the sentiment of wine reviews?

<i>It would still be tricked by sarcasm. I think Twitter and sentiment analyzers are just not a good mix. However, it would be fine for wine reviews.</i>

**Let's say it's a night of political debates.** If I'm trying to report on whether people generally like or dislike what is happening throughout the debates, could I use these sorts of tools on tweets?


<i>You could but you would have to fact-check. I don't think it's going to offer much valuable insight.</i>

We're using the incredibly vague word "okay" on purpose, as there are varying levels of comfort depending on your sitaution. Are you doing this for preliminary research? Are you publishing the results in a journal, in a newspaper, in a report at work, in a public policy recommendation?

What if I tell you that the ideal of "I'd only use a sentiment analysis tool trained exactly for my specific domain" is both _rare and impractical?_ How comfortable do you feel with the output of sentiment analysis if that's the case?

<i>I've not been too impressed by sentiment analysis so my opinion remains the same.</i>

As we saw in the last section, **these tools don't always agree with one another, which might be problematic.**

* What might make them agree or disagree?
* Do we think one is the "best?"
* Can you think of any ways to test which one is the 'best' for our purposes?

<i>
<br> TextBlob + NaiveBayesAnalyzer's word source is pretty limited and Textblob's is slightly less limited, and the results reflect that.
<br> I think NLTK was the best out of the three because it can understand more complex sentences.
<br> It depends on what your objective is. If it's used for analyzing reviews, TextBlob + NaiveBayesAnalyzer and TextBlob would work well. For more complicated analysis, NLTK would be better.
</i>